# 1. 필요한 패키지 불러오기

In [1]:
import numpy as np
import pandas as pd
import time
import re
import random
import joblib
import warnings
warnings.filterwarnings(action='ignore')

from tqdm import tqdm
from collections import namedtuple, Counter
from gensim.models import doc2vec
from konlpy.tag import Okt
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# 2. get_prediction 함수

1. 이 함수는 테스트에 해당하는 csv 파일 경로를 매개변수로 가집니다.
2. 이 함수는 미리 만들어진 모델을 불러와 예측을 수행합니다. 
3. 이 함수는 예측치를 불러온 csv 파일에 넣습니다. 이 때 예측치에 해당하는 column name은 smishing 입니다.
4. 이 함수는 데이터프레임을 리턴합니다.

In [2]:
def get_prediction(test_file_path):
    '''
    Args: String
    Return: Pandas DataFrame    
    '''
    
    '''1. 테스트 파일 불러오기'''
    test_data=pd.read_csv(test_file_path)
    
    
    '''2. 모델 불러오기'''
    lgb = joblib.load('../1_Model/lgb.pkl')
    doc_init = Doc2Vec.load('../1_Model/seven.doc2vec')
    
    '''3. 예측 전 필요한 사항 진행하기'''
    okt = Okt()

    removeword=['[Web발신]', "(광고)", "\n", "\r",'X','XX','XXX','XXXX','XXXXX','XXXXXX','XXXXXXX',
                '.', '을', '를', '이', '가', '-', '(', ')', ':', '!', '?', ')-', '.-', 'ㅡ','..', '.(', '은', '는'] #그냥 사용시 토큰으로 같이 인식됨
    # X를 추가한 이유 : 제거 안하니까 kbXXX이런식으로 같이 인식해버림, 굳이 그렇게 인식 될 이유가 전혀 없어보였음

    def ishangul(text):
        """해당 글자가 한글인지 찾아주는 함수입니다."""
        hanCount = len(re.findall(u'[\u3130-\u318F\uAC00-\uD7A3]+', text))
        return hanCount > 0

    def blank(text):
        """한글과 한글이 아닌 단어사이에 공백을 삽입합니다."""
        return_doc = []
        pre_hangul = True
        for i in text:
            hangul = ishangul(i)
            if pre_hangul != hangul:
                return_doc.append(" ")
                return_doc.append(i)
            else:
                return_doc.append(i)
            pre_hangul = hangul
        return "".join(return_doc)

    def tokenize(doc, norm=True, stem=True, removeword=None):
        """
        공백 삽입을 찾아내고, 이후 doc2vec 을 하기위한 데이터로 바꾸줍니다.
        특수문자를 'Punctuation' 으로 바꿉니다.
        외국어와 숫자를 제거합니다.
        """
        if removeword:
            for word in removeword:
                doc = doc.replace(word, " ") #없앨 단어 대체
        toekns = ["/".join(word) for word in okt.pos(doc)] #토큰 화
        doc = blank(doc)  # 해당 문자에서 영어와 한글 사이에 공백을 넣는 함수

        adj_toekns = []
        for t in toekns:
            if "Punctuation" in t:
                adj_toekns.append("Punctuation")
            elif "Foreign" in t: # 외국어 제거
                pass
            elif "Number" in t:  # 숫자 제거
                pass
            else:
                adj_toekns.append(t) 
        return adj_toekns #외국어, 숫자 제거하고 특수문자는 Punctuation 자체로 바꾸고, url도 해당 url 특징에 맞게 바꾸는 함수


    #위에 주어진 함수를돌리기 위해서 array화 실행

    test_data['smishing']=2
    test_data_list = np.array(test_data[["text","smishing"]].values.tolist())
    test_docs = [ ( tokenize(row[0],  removeword=removeword ), row[1] ) for row in tqdm(test_data_list) ]
    
    TaggedDocument = namedtuple('TaggedDocument', 'words tags')

    tagged_pred_docs = [TaggedDocument(d, [c]) for d, c in test_docs] # d는 단어랑 타입, c는 스미싱 여부 라벨링(test에서는 없으니 임의로 설정된 것)

    def t1(tt):
        doc_init.random.seed(0) #동일한 시드를 놔둬서 같은 array의 값이 나오게 설정해둠
        return doc_init.infer_vector(tt.words) #단어를 vector화 시킴

    pred_x=[ t1(i) for i in tagged_pred_docs ]

    pred_tf = pd.DataFrame(pred_x)
    
    '''4. 예측 진행하기'''
    
    proba= [ i[1] for i in lgb.predict_proba(pred_tf) ]    
    
    '''5. 예측치 데이터프레임에 합치기'''
    
    test_data['smishing']=proba
    
    return test_data

In [3]:
# get_prediction 함수의 return 형식을 보여주기 위한 예제 입니다. 작성하지 않으셔도 무관합니다.
get_prediction('../0_Data/public_test.csv').head()

100%|██████████████████████████████████████████████████████████████████████████████| 1626/1626 [00:23<00:00, 69.28it/s]


,id,year_month,text,smishing
0,340000,2019-01,XXX고객님! 안녕하세요? 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...,6.175360e-05
1,340001,2019-01,긴급 안내 XXX은행 가락동 지점 - 헬리오XXX 기본XXX 대출이자를 ...,8.922148e-07
2,340002,2019-01,XXX 고객님 안녕하세요올해는 미세먼지가 유난인거 같습니다.엊그제 새해가 시작된거같...,8.816719e-05
3,340003,2019-01,XXX 고객님찾아온 행운을 잡으셨나요? 못잡으셨다면 이번에 다시 잡으시길 기원합니다...,3.139068e-06
4,340004,2019-01,XXX 고객님새해 복 많이 받으세요 XXX은행 코스트코 퇴직연금 담당자입니다. 고...,1.406295e-05
